In [3]:
import numpy as np
import pickle
import util

### Load data

In [7]:
datapath = "/home/asd/data/gsm/"
hour10 = util.load_regular_sample_file(datapath, 10)
hour8 = util.load_regular_sample_file(datapath, 8)

### Set up some low rank models

Models worth trying include
- PCA
- NMF
- Sparse variants of PCA and NMF

In [9]:
def fit_nmf(data, k):
    nmf = NMF(n_components=k, init="nndsvd", solver="cd")
    W = nmf.fit_transform(data)
    H = nmf.components_
    return W, H, np.linalg.norm(np.dot(W, H) - data) / np.linalg.norm(data)

In [10]:
W8, H8, loss8 = fit_nmf(hour8, 50)
print("Loss:", loss8)

Loss: 0.2654755924721412


In [11]:
W10, H10, loss10 = fit_nmf(hour10, 50)
print("Loss:", loss10)

Loss: 0.23548096087104697


### Now have these models vary over time

We have data that varies over time, but comes from the same underlying "process." Let's model this as a series of matrix factorizations. Our objective function will be

$$ 
J(W, H) 
= 
\sum_{t=1}^T  \| W^{(t)} H^{(t)} - X^{(t)} \|_F^2 
+ \sum_{t=2}^T  \| H^{(t)} - H^{(t-1)} \|_F^2 
+ R(W, H)
$$

where $R(W,H)$ is an additional regularizer, e.g. nonnegativity, sparsity, etc.

Some questions:
- What if we formulate this as a Bayesian network, where the factors $H$ form a Markov process, but ottherwise everything is the same.
- Is there a natural Bayesian network interpretation of these update rules, which can then be generalized to larger networks?